## Pulling ASO from SnowEx database

This code was built from the SnowEx hackweek database tutorial

### Import stuff to work with database and rasters:

In [1]:
# Import the function to get connect to the db
from snowexsql.db import get_db

# Import our class for the points table
from snowexsql.data import ImageData

# import this to use define sql functions (e.g. postgis!)
from sqlalchemy.sql import func 

# Import this to convert to a rasterio object for easy plotting
from snowexsql.conversions import raster_to_rasterio 

# Import a convenient function to plot with 
from rasterio.plot import show

# Import
from geoalchemy2.types import Raster
import geoalchemy2.functions as gfunc

import geopandas as gpd
import matplotlib.pyplot as plt

from sqlalchemy.sql import func
import shapely.geometry
from geoalchemy2.shape import from_shape
import geoalchemy2.functions as gfunc

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

In [2]:
# Import the function to investigate a table
from snowexsql.db import get_table_attributes

# Use the function to see what columns are available to use. 
db_columns = get_table_attributes(ImageData)

# Print out the results nicely
print("These are the available columns in the table:\n \n* {}\n".format('\n* '.join(db_columns)))

These are the available columns in the table:
 
* date
* date_accessed
* description
* doi
* instrument
* metadata
* observers
* raster
* registry
* site_name
* time_created
* time_updated
* type
* units



### What to filter?

In [ ]:
# This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name)

# Get the unique datanames in the table
results = session.query(ImageData.type).distinct().all()
print('Available types = {}'.format(', '.join([r[0] for r in results])))

# Get the unique instrument in the table
results = session.query(ImageData.instrument).distinct().all()
print('\nAvailable Instruments = {}'.format(', '.join([str(r[0]) for r in results])))

# Get the unique dates in the table
results = session.query(ImageData.date).distinct().all()
print('\nAvailable Dates = {}'.format(', '.join([str(r[0]) for r in results])))

# Get the unique sites in the table
results = session.query(ImageData.site_name).distinct().all()
print('\nAvailable sites = {}'.format(', '.join([str(r[0]) for r in results])))

# Get the unique surveyors in the table
results = session.query(ImageData.observers).distinct().all()
print('\nAvailable surveyors = {}'.format(', '.join([str(r[0]) for r in results])))

session.close()

### Query

In [ ]:
# Read in shapefile
# shp = gpd.read_file('~/Study_sites/Study-sites.shp')

## From Mia: 
#snagged these from somewhere (sliderule GM notebook?), but would replace
#with bounds from uavsar rasters/elsewhere
gm_bds = [(39.34603060272382, -108.40601489205419),
          (39.32770853617356, -107.68485163209928),
          (39.32770853617356, -107.68485163209928),
          (38.788639821001155, -108.42635020791396),
          (39.34603060272382, -108.40601489205419)]
gm_poly = shapely.geometry.Polygon(gm_bds)
#converting shapefile to srid 26912 (UTM 12N) for db querying:
import pyproj
from shapely.ops import transform
wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:26912')
project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
utm_poly = transform(project, gm_poly)
# wkb_element = from_shape(utm_poly, srid=26912)
#basic bbox, since not getting anything w/ complex polygon object:
#(count = 0) after query
w,s,e,n= -108.42635020791396, 38.788639821001155, -107.68485163209928, 39.34603060272382
bbox = shapely.geometry.Polygon([[w, s], [w, n], [e, n], [e, s]])
bbox = transform(project, bbox)
bbox_wkb = from_shape(bbox, srid=26912)
##


# Grab a session
engine, session = get_db(db_name)

# 3. Merge all the tiles together, note gfunc vs func. This is because ST_Union exists in two places in postgis for geom and rasters!
base = gfunc.ST_Union(ImageData.raster, _type=Raster)

# 4. Get the merged result as a geotiff! 
base = func.ST_AsTiff(base)

# 5. Filter by lidar data
qry = session.query(base).filter(ImageData.type == 'DEM')

# 7. Get rasters within shapefile
qry = session.query(func.ST_Clip(ImageData.raster, bbox_wkb))

print(qry.count())

# 8. Execute, convert and plot! 
result = qry.all()
datasets = raster_to_rasterio(session, result)
show(datasets[0], cmap='Purples')

session.close()



In [ ]:
# import our pits metadata table class
from snowexsql.data import SiteData
from geoalchemy2.types import Raster
import geoalchemy2.functions as gfunc

# Grab a session
engine, session = get_db(db_name)

# session.rollback()

# 1. Lets choose a site we want to grab a raster tile
site_id = '5S31'

# 2. Get the location of the pit, POSTGIS functions like to work in the text format of things so convert the point geom to text which is also in binary in the db   
point = session.query(SiteData.geom).filter(SiteData.site_id == site_id).distinct().all()[0][0]

# 3. Merge all the tiles together, note gfunc vs func. This is because ST_Union exists in two places in postgis for geom and rasters!
base = gfunc.ST_Union(ImageData.raster, _type=Raster)

# 4. Get the merged result as a geotiff! 
base = func.ST_AsTiff(base)

# 5. Filter by lidar data
qry = session.query(base).filter(ImageData.type == 'DEM')

# 7. Isolate tiles touching the pit location
qry = qry.filter(func.ST_Intersects(ImageData.raster, point))

print(qry.count())

# 8. Execute, convert and plot! 
result = qry.all()
datasets = raster_to_rasterio(session, result)
show(datasets[0], cmap='Purples')
#plt.colorbar()

session.close()

In [ ]:
datasets[0].read()